In [1]:
from pandas.tseries.offsets import DateOffset
from utils.reader import ReadData
import pandas as pd

def train_test_split(df):

    def set_datetime(df):
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date', drop=True, inplace=False)
        df.sort_index(inplace=True)
        return df

    def get_split(df: pd.DataFrame):
        offset = df.index[-1] - DateOffset(weeks=1)
        df_train = df[df.index < offset]
        df_test = df[df.index >= offset]
        df_train.reset_index(inplace=True)
        df_test.reset_index(inplace=True)
        return df_train, df_test

    df = set_datetime(df)
    df_train, df_test = get_split(df)

    return df_train, df_test

def set_target(df):
    TARGET = 'TARGET'
    qp, qr = 'quantity_of_paid', 'quantity_of_return',
    df[TARGET] = df[qp] - df[qr]
    df.drop([qp, qr], axis=1, inplace=True)
    return df

def load_and_preprocess(file):
    # data, feature, meta
    reader = ReadData()
    df = reader.read(file)
    if file == 'data':
        df = set_target(df)
    df_train, df_test = train_test_split(df)
    return df_train, df_test


df_data_train, df_data_test = load_and_preprocess('data')

In [2]:
from pipeline.step_0.build_pipeline import load_and_preprocess as load_and_preprocess_step_0
df_train = load_and_preprocess_step_0(df_data_train, inference=False)
df_test = load_and_preprocess_step_0(df_data_test, inference=True)

[Pipeline] ......... (step 1 of 6) Processing drop_data, total=   0.0s
[Pipeline] . (step 2 of 6) Processing drop_na_base_data, total=   0.0s
[Pipeline] . (step 3 of 6) Processing drop_fill_na_data, total=   0.1s
[Pipeline]  (step 4 of 6) Processing drop_non_informative_data, total=   0.0s
[Pipeline]  (step 5 of 6) Processing join_non_informative_data, total=   0.0s
[Pipeline] . (step 6 of 6) Processing group_by_day_data, total=   4.1s


/Users/dev.daniil.bakushkin/Desktop/suppi/ml/app/pipeline/step_0/custom_transformers.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col].fillna(mode_, inplace=True)
/Users/dev.daniil.bakushkin/Desktop/suppi/ml/app/pipeline/step_0/custom_transformers.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col].fillna(X.groupby([self.col_group_by_1, self.col_group_by_2])[
/Users/dev.daniil.bakushkin/Desktop/suppi/ml/app/pipeline/step_0/custom_transformers.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

In [7]:
# from pipeline.step_1.build_pipeline import load_and_preprocess as load_and_preprocess_step_1

from pipeline.step_1 import build_pipeline
import imp
imp.reload(build_pipeline)

load_and_preprocess_step_1 = build_pipeline.load_and_preprocess
df_feature_train, df_feature_test = load_and_preprocess('feature')
# df_feature_train = df_feature_train.loc[:100, :]
# df_feature_test = df_feature_test.loc[:100, :]

df_feature_train = load_and_preprocess_step_1(
    df_feature_train, inference=False)
df__feature_test = load_and_preprocess_step_1(df_feature_test, inference=True)

[Pipeline] ...... (step 1 of 2) Processing keep_feature, total=   0.0s


/Users/dev.daniil.bakushkin/miniconda3/envs/shad/lib/python3.11/site-packages/sklearn/utils/extmath.py:1051: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/dev.daniil.bakushkin/miniconda3/envs/shad/lib/python3.11/site-packages/sklearn/utils/extmath.py:1056: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/dev.daniil.bakushkin/miniconda3/envs/shad/lib/python3.11/site-packages/sklearn/utils/extmath.py:1076: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


[Pipeline] ............... (step 2 of 2) Processing knn, total=  33.2s


In [8]:
for col in df_feature_train.columns:
  print(df_feature_train[col].nunique(), col)

93 date
499 offer_id
1 shop_id
510 num_actions
2860 position_category
1 price_index
1 external_index_data_minimal_price
1 external_index_data_price_index_value
1 self_marketplaces_index_data_minimal_price
1 self_marketplaces_index_data_price_index_value
1 marketing_seller_price
234 marketing_price
1 conv_tocart
1386 conv_tocart_pdp
1269 hits_tocart
1 hits_tocart_pdp
1 hits_view_search


In [9]:
for col in df_feature_train.columns:
  print(df_feature_train[col].nunique(), col)

93 date
499 offer_id
1 shop_id
510 num_actions
2860 position_category
1 price_index
1 external_index_data_minimal_price
1 external_index_data_price_index_value
1 self_marketplaces_index_data_minimal_price
1 self_marketplaces_index_data_price_index_value
1 marketing_seller_price
234 marketing_price
1 conv_tocart
1386 conv_tocart_pdp
1269 hits_tocart
1 hits_tocart_pdp
1 hits_view_search
